In [156]:
import pandas as pd
import regex as re

In [161]:
data = pd.read_csv('skolkovo_hack/log_week_end.csv', names=['rn', 'loguser', 'q'])

C:\Users\darkw\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [162]:
data.sample(5)

,rn,loguser,q
190336,322334,etl_2048,from tbl_132974
128666,956971,etl_1151,from tbl_21345
58573,139142,etl_2048,from tbl_140657
531663,723571,dev_1751,from tbl_199786
220584,691001,etl_2048,"into tbl_178564,join tbl_178564,from tbl_178565"


Собираем в списки все команды

In [164]:
intos, froms, joins = [], [], []

In [165]:
for x in data.q.str.split(','):
    for g in x:
        if 'from' in g:
            froms.append(g)
        if 'join' in g:
            joins.append(g)
        if 'into' in g:
            intos.append(g)

Оставляем только название таблиц

In [166]:
intos_tbs = [re.search('tbl_\d*', x).group(0) for x in intos]
froms_tbs = [re.search('tbl_\d*', x).group(0) for x in froms]
joins_tbs = [re.search('tbl_\d*', x).group(0) for x in joins]

Делаем датафрейм с окличество into и from (from только от пользователей dev)

In [167]:
df1 = pd.DataFrame(data = froms_tbs, columns=['col']).groupby('col').agg({'col': 'count'})
df1.columns= ['count_froms']
df1.reset_index(inplace=True)

In [168]:
df1

,col,count_froms
0,tbl_10000,24
1,tbl_10001,24
2,tbl_100026,6
3,tbl_100027,6
4,tbl_100028,6
...,...,...
34774,tbl_9996,24
34775,tbl_9997,24
34776,tbl_9998,24
34777,tbl_99986,20


In [169]:
df2 = pd.DataFrame(data = intos_tbs, columns=['col']).groupby('col').agg({'col': 'count'})
df2.columns= ['count_intos']
df2.reset_index(inplace=True)
df2

,col,count_intos
0,tbl_10000,12
1,tbl_10001,12
2,tbl_100026,6
3,tbl_100027,6
4,tbl_100028,6
...,...,...
25240,tbl_9995,12
25241,tbl_9996,12
25242,tbl_9997,12
25243,tbl_9998,12


In [171]:
df = df1.merge(df2, on ='col', how='outer')

In [177]:
df['usefull'] = df.count_intos / df.count_froms
df.sort_values(by='usefull', ascending = False)

,col,count_froms,count_intos,usefull
18715,tbl_211513,8.0,9593.0,1199.125000
21142,tbl_29455,1.0,470.0,470.000000
20585,tbl_26776,1.0,468.0,468.000000
20643,tbl_27182,1.0,464.0,464.000000
18714,tbl_211512,21.0,3138.0,149.428571
...,...,...,...,...
41267,tbl_97783,NaN,21.0,NaN
41268,tbl_97784,NaN,21.0,NaN
41269,tbl_99127,NaN,6.0,NaN
41270,tbl_99134,NaN,6.0,NaN
